##  >   Extract bussiness insights and visulaize our data

In [1]:
# import visulaization indpendecies
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly as pt

In [2]:
## import general indpendecies 
import pandas as pd
import json
import csv
import numpy as np
import psycopg2
import ast
from sqlalchemy import create_engine, text

In [3]:
## connect to db
# connnect to db 
# Replace these with your database details
dbname = "nawy"
user = "nady"
password = "admin"
host = "172.20.0.3"
port = "5432"


# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

except psycopg2.Error as e:
    print("Error connecting to the database:", e)
    exit()


# Create SQLalchemy engine
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

In [17]:
# Query to retrieve all property with each value in map
with engine.connect() as connection:
    query = text("""
                     SELECT c.compound_id, e.area_name, e.lat, e.long
                    FROM compound as c inner join area as e
                    ON c.area_id = e.area_id""")
    results = connection.execute(query)

    # Fetch the results
    result = results.fetchall()
    field_name = results.keys()


In [18]:
import plotly.express as px

# Unpack the data into separate lists
compounds_ids, areaName, latitudes, longitudes = zip(*result)

# Use Plotly Express to create a scatter map
fig = px.scatter_mapbox(lat=latitudes,
                        lon=longitudes,
                        hover_data={'compoundsID': compounds_ids, 'areaName': areaName},
                        zoom=10)

# Customize the map layout if needed
fig.update_layout(mapbox_style='open-street-map',
                  margin=dict(l=0, r=0, t=0, b=0))

# Show the plot
fig.show()


In [79]:
## get property sorted by max instalment years in "Ras Sudr" commpoundId = 710

with engine.connect() as connection:
    query = text("""
                    SELECT 
                        "propertyName", "propertyID",
                        max_installment_years,min_down_payment, compound_id
                    FROM 
                        property
                    WHERE max_installment_years IS NOT NULL
                    ORDER BY max_installment_years DESC""")
    results = connection.execute(query)

    # Fetch the results
    result = results.fetchall()
    field_name = results.keys()
print(result)

[('Chalet - BRELLIA - for sale in The Groove with 3 bedrooms in Ain Sokhna by DM DEVELOPMENT', 35815, 25.0, Decimal('236500'), 392), ('Apartment for sale in Il Latini SED with 2 bedrooms in North Coast by Saudi Egyptian Developers (SED)', 42083, 10.0, Decimal('826000'), 493), ('Apartment for sale in Il Latini SED with 2 bedrooms in North Coast by Saudi Egyptian Developers (SED)', 41420, 10.0, Decimal('819200'), 493), ('Apartment for sale in Il Latini SED with 2 bedrooms in North Coast by Saudi Egyptian Developers (SED)', 42395, 10.0, Decimal('803200'), 493), ('Apartment for sale in Il Latini SED with 1 bedroom in North Coast by Saudi Egyptian Developers (SED)', 42079, 10.0, Decimal('793800'), 493), ('Apartment for sale in Il Latini SED with 2 bedrooms in North Coast by Saudi Egyptian Developers (SED)', 42086, 10.0, Decimal('787200'), 493), ('Apartment for sale in Il Latini SED with 2 bedrooms in North Coast by Saudi Egyptian Developers (SED)', 40426, 10.0, Decimal('775400'), 493), ('Ap

In [80]:
# Unpack the data into separate lists
from prettytable import PrettyTable
# Unpack the data into separate lists
property_names, property_ids, max_installment_years, min_down_payment,  compound_ids = zip(*result)

# Create a PrettyTable instance and add columns
table = PrettyTable()
table.field_names = ['PropertyDesc', 'PropertyID', 'MaxInstallmentYears', 'min_down_payment', 'CompoundID']

# Add rows to the table
for row in zip(property_names, property_ids, max_installment_years, min_down_payment, compound_ids):
    table.add_row(row)

# Print the table
print(table)

+--------------------------------------------------------------------------------------------------------------------------------------+------------+---------------------+------------------+------------+
|                                                             PropertyDesc                                                             | PropertyID | MaxInstallmentYears | min_down_payment | CompoundID |
+--------------------------------------------------------------------------------------------------------------------------------------+------------+---------------------+------------------+------------+
|                      Chalet - BRELLIA - for sale in The Groove with 3 bedrooms in Ain Sokhna by DM DEVELOPMENT                       |   35815    |         25.0        |      236500      |    392     |
|                Apartment for sale in Il Latini SED with 2 bedrooms in North Coast by Saudi Egyptian Developers (SED)                 |   42083    |         10.0        |      826000 

In [67]:
## visulaize property as scatter with it's ranking

with engine.connect() as connection:
    query = text("""
                    SELECT 
                        "propertyID", "manual_ranking"
                    FROM 
                        property
                    --WHERE manual_ranking IS NOT NULL
                    ORDER BY max_installment_years DESC""")
    results = connection.execute(query)

    # Fetch the results
    result = results.fetchall()
    field_name = results.keys()

In [68]:
## visulaize propertyids with manual ranking
import plotly.express as px


# Unpack the data into separate lists and cast 'propertyID' to integers
property_ids, manual_rankings = zip(*[(int(prop_id), rank) for prop_id, rank in result])

# Create a scatter plot with 'category' x-axis
fig = px.scatter(x=property_ids, y=manual_rankings, labels={'x': 'PropertyID', 'y': 'Manual Ranking'},
                 title='Scatter Diagram of Property Manual Rankings')

fig.update_layout(xaxis_type='category')

# Show the plot
fig.show()


In [72]:
## SELECT properties has limitted time offer

with engine.connect() as connection:
    query = text("""
                    SELECT 
                        "propertyID"
                    FROM 
                        property
                    WHERE limited_time_offer = 'True'
                    """)
    results = connection.execute(query)

    # Fetch the results
    result = results.fetchall()
    field_name = results.keys()
print(result)

[]


### >> No properties has limited time offer

In [86]:
## SELECT top 100 properties with high completeness_score and where it located

with engine.connect() as connection:
    query = text("""
                    SELECT 
                        p."propertyID", p.compound_id, a.area_name
                    FROM 
                        property as p inner join  area as a
                    ON
                        p."areaID" = a.area_id
                    ORDER BY  completeness_score DESC
                    limit 100""")
    results = connection.execute(query)

    # Fetch the results
    result = results.fetchall()
    field_name = results.keys()

In [87]:
# Unpack the data into separate lists
from prettytable import PrettyTable
propertyId, compoundId, area_name = zip(*result)

# Create a PrettyTable instance and add columns
table = PrettyTable()
table.field_names = ['propertyId', 'compoundId', 'area_name']

# Add rows to the table
for row in zip(propertyId, compoundId, area_name):
    table.add_row(row)

# Print the table
print(table)

+------------+------------+---------------------+
| propertyId | compoundId |      area_name      |
+------------+------------+---------------------+
|   39397    |     90     |      Ain Sokhna     |
|   39396    |     90     |      Ain Sokhna     |
|   42496    |    222     |      New Cairo      |
|   42443    |    222     |      New Cairo      |
|   31767    |     90     |      Ain Sokhna     |
|    4300    |    458     |   New Capital City  |
|    7941    |    503     |   New Capital City  |
|   39889    |    345     |      New Cairo      |
|   33445    |    222     |      New Cairo      |
|   42440    |    222     |      New Cairo      |
|   38921    |     90     |      Ain Sokhna     |
|   40912    |     90     |      Ain Sokhna     |
|   35495    |    195     |      New Cairo      |
|   35497    |    195     |      New Cairo      |
|   35490    |    195     |      New Cairo      |
|   35487    |    195     |      New Cairo      |
|   35486    |    195     |      New Cairo      |


In [98]:
## get top 5 developer has finshed units
## SELECT top 100 properties with high completeness_score and where it located

with engine.connect() as connection:
    query = text("""
                   SELECT 
	                    d.developer_name, count(*) as count_of_property
                    FROM 
                        property as p inner join  developer as d
                    ON
                        p.developer_id = d.developer_id
                    WHERE
                        p.finishing = 'finished'
                    GROUP BY 1
                    ORDER BY 2 DESC """)
    results = connection.execute(query)

    # Fetch the results
    result = results.fetchall()
    field_name = results.keys()

In [100]:
import plotly.express as px

# Unpack the data into separate lists
developers, project_counts = zip(*result)

# Create a bar chart
fig = px.bar(x=developers, y=project_counts, labels={'x': 'Developer', 'y': 'Project_finshing_counts'},
             title='Number of Projects by Developer')

# Show the plot
fig.show()
